In [1]:
!pip install antigranular


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.6 MB/s eta 0:00:00


In [61]:
import antigranular as ag
session = ag.login("9CyMo7mF8P2ZK9LSyT0Z308K1CpwwiGj", "05UP78uWcFCCwPJsxWBLkeo48F2HeOXC9+TSy/5hMFyto0SZK5YzKU6CPmfvvo89", competition = "Harvard OpenDP Hackathon")

Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Connected to Antigranular server session id: ad507198-09a1-4450-8d85-aed9c8683599, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [62]:
%%ag
health = health_organisation_dataset.copy()
flight = flight_company_dataset.copy()
# Lets remove those passenger records who tested negative.
health['covidtest_result'] = health['covidtest_result'].where(health['covidtest_result'] == 'positive')
health = health.dropna()

In [63]:
%%ag
ag_print(health.dtypes)

patient_firstname        object
patient_lastname         object
patient_date_of_birth    object
patient_phone_number     object
patient_email_address    object
covidtest_date           object
covidtest_result         object
patient_address          object
dtype: object



In [64]:
%%ag
ag_print(flight.dtypes)

index                       int64
flight_number              object
flight_date                object
flight_from                object
flight_to                  object
passenger_firstname        object
passenger_lastname         object
passenger_date_of_birth    object
passenger_phone_number     object
passenger_email_address    object
dtype: object



In [65]:
%%ag
import re
#import pandas as pd
from op_pandas import PrivateDataFrame
#from op_pandas import PrivateDataFrame
# Updating the function to handle numeric month formats (e.g., "02/15/78") as well
def extract_month(date_str:str)->str:
    month_conversion = {
        "jan": "01", "january": "01",
        "feb": "02", "february": "02",
        "mar": "03", "march": "03",
        "apr": "04", "april": "04",
        "may": "05",
        "jun": "06", "june": "06",
        "jul": "07", "july": "07",
        "aug": "08", "august": "08",
        "sep": "09", "september": "09",
        "oct": "10", "october": "10",
        "nov": "11", "november": "11",
        "dec": "12", "december": "12"
    }
    month_conversion.update({k.capitalize(): v for k, v in month_conversion.items()})
    # First, try to find a month name or abbreviation
    for month_name, month_number in month_conversion.items():
        if month_name in date_str:
            return month_number

    # If no month name is found, try to extract numeric month format
    numeric_month_match = re.search(r'\b(0[1-9]|1[0-2])\b', date_str)
    if numeric_month_match:
        return numeric_month_match.group()

    return date_str
# Function to extract the day from various date formats
def extract_day(date_str:str)->str:
    # Regular expression to match a day format (1 or 2 digits, possibly leading with 0)
    day_match = re.search(r'\b(0?[1-9]|[12][0-9]|3[01])\b', date_str)
    if day_match:
        return day_match.group()

    return date_str
def add_leading_zero(day:str)->str:
    if (len(day)>1):
      return day
    else:
      return f"0{day}"




# Function to extract the year
def extract_year_19(date_str):
    # Regular expression to match a 2 or 4 digit year
    matched = re.search(r'(\d{2}|\d{4})$', date_str)
    if matched:
        # Extract the year
        year = matched.group(0)
        # Handling 2-digit years (assuming 20th century)
        if len(year) == 2:
          if int(year)>23:
            year = '19' + year
          else:
            year = '20' + year
        return year
    else:
        return date_str

# def extract_year_20(date_str):
#     # Regular expression to match a 2 or 4 digit year
#     matched = re.search(r'(\d{2}|\d{4})$', date_str)
#     if matched:
#         # Extract the year
#         year = matched.group(0)
#         # Handling 2-digit years (assuming 20th century)
#         if len(year) == 2:
#             year = '20' + year
#         return year
#     else:
#         return date_str


def full_df(df: PrivateDataFrame, col: str) -> None:
  df[col+'month'] = df[col].map(extract_month)
  df[col+'day'] = df[col].map(extract_day).map(add_leading_zero)
  df[col+'year19'] = df[col].map(extract_year_19)
  #df[col+'year20'] = df[col].map(extract_year_20)

  df[col+'fulldate'] = df[col+'month']+df[col+'day']+df[col+'year19']

# liste = ['December 14 2008', '23 Sep 94', 'Mar 03 84', '23 December 79',
#          'October-07-71', '12-May-1976', '19-Aug-76', '01-Nov-2007',
#          '07-September-1989', 'Sep 25 66', '27.April.1973', 'Apr.10.11',
#          'April 04 86']

# # Creating a DataFrame
# df = pd.DataFrame(liste, columns=['date'])

# full_df(df,'date')

full_df(flight,'passenger_date_of_birth')
full_df(health,'patient_date_of_birth')
# full_df(flight,'flight_date')
# full_df(health,'covidtest_date')

In [115]:
# %%ag
# import re
# #import pandas as pd
# from op_pandas import PrivateDataFrame
# #from op_pandas import PrivateDataFrame
# # Updating the function to handle numeric month formats (e.g., "02/15/78") as well
# def extract_month(date_str:str)->str:
#     month_conversion = {
#         "jan": "01", "january": "01",
#         "feb": "02", "february": "02",
#         "mar": "03", "march": "03",
#         "apr": "04", "april": "04",
#         "may": "05",
#         "jun": "06", "june": "06",
#         "jul": "07", "july": "07",
#         "aug": "08", "august": "08",
#         "sep": "09", "september": "09",
#         "oct": "10", "october": "10",
#         "nov": "11", "november": "11",
#         "dec": "12", "december": "12"
#     }
#     month_conversion.update({k.capitalize(): v for k, v in month_conversion.items()})
#     # First, try to find a month name or abbreviation
#     for month_name, month_number in month_conversion.items():
#         if month_name in date_str:
#             return month_number

#     # If no month name is found, try to extract numeric month format
#     numeric_month_match = re.search(r'\b(0[1-9]|1[0-2])\b', date_str)
#     if numeric_month_match:
#         return numeric_month_match.group()

#     return date_str
# # Function to extract the day from various date formats
# def extract_day(date_str:str)->str:
#     # Regular expression to match a day format (1 or 2 digits, possibly leading with 0)
#     day_match = re.search(r'\b(0?[1-9]|[12][0-9]|3[01])\b', date_str)
#     if day_match:
#         return day_match.group()

#     return date_str
# def add_leading_zero(day:str)->str:
#     if (len(day)>1):
#       return day
#     else:
#       return f"0{day}"




# # Function to extract the year
# def extract_year_19(date_str:str)->str:

#     """
#     Extract the last two digits of the year from a date string.
#     If the year is 4 digits, returns the last two digits.
#     If the year is 2 digits, returns those two digits.
#     """
#     # Regular expression to match a 2 or 4 digit year
#     matched = re.search(r'(\d{2}|\d{4})$', date_str)
#     if matched:
#         # Extract the year
#         year = matched.group(0)
#         # Return the last two digits
#         return year[-2:]
#     else:
#         return date_str


# # def extract_year_20(date_str):
# #     # Regular expression to match a 2 or 4 digit year
# #     matched = re.search(r'(\d{2}|\d{4})$', date_str)
# #     if matched:
# #         # Extract the year
# #         year = matched.group(0)
# #         # Handling 2-digit years (assuming 20th century)
# #         if len(year) == 2:
# #             year = '20' + year
# #         return year
# #     else:
# #         return date_str


# def full_df(df: PrivateDataFrame, col: str) -> None:
#   df[col+'month'] = df[col].map(extract_month)
#   df[col+'day'] = df[col].map(extract_day).map(add_leading_zero)
#   df[col+'year19'] = df[col].map(extract_year_19)
#   #df[col+'year20'] = df[col].map(extract_year_20)

#   df[col+'fulldate'] = df[col+'month']+df[col+'day']+df[col+'year19']

# # liste = ['December 14 2008', '23 Sep 94', 'Mar 03 84', '23 December 79',
# #          'October-07-71', '12-May-1976', '19-Aug-76', '01-Nov-2007',
# #          '07-September-1989', 'Sep 25 66', '27.April.1973', 'Apr.10.11',
# #          'April 04 86']

# # # Creating a DataFrame
# # df = pd.DataFrame(liste, columns=['date'])

# # full_df(df,'date')

# full_df(flight,'passenger_date_of_birth')
# full_df(health,'patient_date_of_birth')
# # full_df(flight,'flight_date')
# # full_df(health,'covidtest_date')

In [116]:
# 4 haneli ise 2 hane çevir
#day month year ,
#first last name
# full index yapsak ne değişir

In [16]:
%%ag
ag_print(health.columns)

['patient_firstname', 'patient_lastname', 'patient_date_of_birth', 'patient_phone_number', 'patient_email_address', 'covidtest_date', 'covidtest_result', 'patient_address', 'patient_date_of_birthmonth', 'patient_date_of_birthday', 'patient_date_of_birthyear19', 'patient_date_of_birthfulldate', 'covidtest_datemonth', 'covidtest_dateday', 'covidtest_dateyear19', 'covidtest_datefulldate']



In [17]:
%%ag
ag_print(flight.columns)

['index', 'flight_number', 'flight_date', 'flight_from', 'flight_to', 'passenger_firstname', 'passenger_lastname', 'passenger_date_of_birth', 'passenger_phone_number', 'passenger_email_address', 'passenger_date_of_birthmonth', 'passenger_date_of_birthday', 'passenger_date_of_birthyear19', 'passenger_date_of_birthfulldate', 'flight_datemonth', 'flight_dateday', 'flight_dateyear19', 'flight_datefulldate']



In [66]:
%%ag

import numpy as np
import op_pandas as opd
import pandas as pd
flight['passenger_date_of_birth'] = op_pandas.to_datetime(flight['passenger_date_of_birth'],errors='coerce')
flight['flight_date'] = op_pandas.to_datetime(flight['flight_date'],errors='coerce')
health['patient_date_of_birth'] = op_pandas.to_datetime(health['patient_date_of_birth'], errors='coerce')
health['covidtest_date'] = op_pandas.to_datetime(health['covidtest_date'], errors='coerce')

In [67]:
%%ag

import numpy as np
import op_pandas as opd
import pandas as pd
flight['passenger_date_of_birthfulldate'] = op_pandas.to_datetime(flight['passenger_date_of_birthfulldate'],format='%m%d%Y',errors='coerce')
#flight['flight_date'] = op_pandas.to_datetime(flight['flight_date'],errors='coerce')
health['patient_date_of_birthfulldate'] = op_pandas.to_datetime(health['patient_date_of_birthfulldate'],format='%m%d%Y', errors='coerce')
#health['covidtest_date'] = op_pandas.to_datetime(health['covidtest_date'], errors='coerce')

In [9]:
%%ag
ag_print(health.dtypes)

patient_firstname                        object
patient_lastname                         object
patient_date_of_birth            datetime64[ns]
patient_phone_number                     object
patient_email_address                    object
covidtest_date                   datetime64[ns]
covidtest_result                         object
patient_address                          object
patient_date_of_birthmonth               object
patient_date_of_birthday                 object
patient_date_of_birthyear19              object
patient_date_of_birthfulldate    datetime64[ns]
dtype: object



In [10]:
%%ag
ag_print(flight.dtypes)

index                                       int64
flight_number                              object
flight_date                        datetime64[ns]
flight_from                                object
flight_to                                  object
passenger_firstname                        object
passenger_lastname                         object
passenger_date_of_birth            datetime64[ns]
passenger_phone_number                     object
passenger_email_address                    object
passenger_date_of_birthmonth               object
passenger_date_of_birthday                 object
passenger_date_of_birthyear19              object
passenger_date_of_birthfulldate    datetime64[ns]
dtype: object



In [ ]:
index                                       int64
flight_number                              object
flight_date                        datetime64[ns]
flight_from                                object
flight_to                                  object
passenger_firstname                        object
passenger_lastname                         object
passenger_date_of_birth            datetime64[ns]
passenger_phone_number                     object
passenger_email_address                    object
passenger_date_of_birthmonth               object
passenger_date_of_birthday                 object
passenger_date_of_birthyear19              object
passenger_date_of_birthfulldate    datetime64[ns]
dtype: object


In [68]:
%%ag
import op_recordlinkage as rl
indexer = rl.Index()

#indexer.block(['passenger_date_of_birth'],['patient_date_of_birth'])
indexer.block(['passenger_date_of_birthfulldate'],['patient_date_of_birthfulldate'])

#indexer.block('passenger_phone_number','patient_phone_number')
#indexer.block('passenger_email_address','patient_email_address')
#indexer.block(['passenger_firstname','passenger_lastname'],['patient_firstname','patient_lastname'])

candidate_links = indexer.index(flight,health)

In [71]:
%%ag
compare = rl.Compare()

compare.exact('passenger_date_of_birthday', 'patient_date_of_birthday',label="day")
compare.string('passenger_date_of_birthmonth', 'patient_date_of_birthmonth',method='jarowinkler', threshold=0.4,label="month")
#compare.string('passenger_date_of_birthyear19', 'patient_date_of_birthyear19',method='jarowinkler', threshold=0.4,label="year")
compare.string("passenger_lastname","patient_lastname" ,method='jarowinkler', threshold=1, label="last")
compare.string("passenger_firstname","patient_firstname" ,method='jarowinkler', threshold=0.75, label="first")
compare.string("passenger_phone_number","patient_phone_number" ,method='levenshtein', threshold=0.1, label="phone")
#compare.string("passenger_phone_number","patient_phone_number" ,method='jarowinkler', threshold=0.3, label="mail")
#compare.string("y","y" ,method='jarowinkler', threshold=0.75, label="pho")
import datetime
def cmp(date_str1:datetime.datetime , date_str2:datetime.datetime)->int: # datetime and regex are pre-imported in isolated environment.
    # Convert date strings to datetime objects
    #date1 = datetime.datetime.strptime(date_str1, "%Y-%m-%d")
    #date2 = datetime.datetime.strptime(date_str2, "%Y-%m-%d")

    # Calculate the absolute difference in days
    days_apart = abs((date_str2 - date_str1).days)
    return 1 if (days_apart <= 13)  else 0
#flight_address,patient_address
# Here's a basic custom function to compare names
# Here's a basic custom function to compare names
def cmp2( name1 : str , name2 : str ) -> float:
  if not isinstance(name1,str):
    return 0.0
  if not isinstance(name2,str):
    return 0.0
  # Convert both names to lowercase for a case-insensitive comparison
  name1 = name1.lower()
  name2 = name2.lower()

  # Exact match
  if name1 == name2:
    return 1.0

  # Initial check: E.g. "A. Smith" and "Adam Smith"
  if name1[0] == name2[0] and (name1[1] == '.' or name2[1] == '.'):
    return 0.9

  # Substring check: E.g. "Rob" and "Robert"
  if name1 in name2 or name2 in name1:
    return 0.8

  # Length difference: E.g. "Ann" and "Anne"
  if abs(len(name1) - len(name2)) == 1:
    return 0.7

  return 0.0


#compare.custom(cmp2,left_on='y',right_on='y',label='name_compare')




compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")



In [73]:


%%ag
features = compare.compute(candidate_links,flight,health)

linked_df = compare.get_match(5.5)


# Submitting the column containing the filtered set of airlines we should report regarding a covid passenger.
res = linked_df[["l_flight_number"]]
x = submit_predictions(res)

score: {'leaderboard': 0.9503180237161766, 'logs': {'LIN_EPS': -0.006, 'MCC': 0.9563180237161766}}



In [ ]:
0.93799

In [140]:
session.terminate_session()

{'status': 'ok'}